In [1]:
from dataset_io import *
from color_channels import equalize_hist
import os
import cv2
import numpy as np
import random as ra
from tqdm import tqdm
from profiler import *
import random
from PIL import Image, ImageCms

In [2]:
!pip install pyCMS

ERROR: Could not find a version that satisfies the requirement pyCMS (from versions: none)
ERROR: No matching distribution found for pyCMS
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
def imread_resized(scene_name, filename, label_resolution):
    filepath=get_filepath(scene_name, filename)
    image = cv2.imread(filepath)
    sh=image.shape
    sh=(int(sh[1]/3), int(sh[0]/3))
    return cv2.resize(image, sh)

In [4]:
def convert_to_gradient_channels(img):
    channels=[]
    
    img = cv2.UMat(img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY )
    channels.append(cv2.UMat(gray))
    
    gray = cv2.GaussianBlur(gray, (3,3), sigmaX=0, sigmaY=0)
    channels.append(cv2.UMat(gray))
    
    gray = cv2.equalizeHist(gray) #experimental
    gradX = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3, scale=1 )
    gradY = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3, scale=1 )
    mag, ori = cv2.cartToPolar(gradX, gradY, angleInDegrees=True)

    ori_ranges = [(0,60), (60,120), (120,180), (180,240), (240,300), (300,360)]
    oris=[cv2.inRange(ori,l,r) for (l,r) in ori_ranges]
    mags= [cv2.copyTo(mag, ori_x) for ori_x in oris]
    channels.extend([gray,mag]+mags)
    channels=[cv2.UMat.get(ch) for ch in channels]
    print(np.array(channels).shape)
    #channels=np.moveaxis(channels, 0, -1)
    return channels

In [5]:
def pil_bgr_to_cmyk(bgr):
    img = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    img = ImageCms.profileToProfile(img, 'color_profiles/AdobeRGB1998.icc', 'color_profiles/USWebUncoated.icc', outputMode='CMYK')
#     img = img.convert('CMYK')
    return np.asarray(img)

def to_color_channels(bgr):
    channels=[]
    
    bgr=equalize_hist(bgr)
    print(np.array(bgr).shape)
    
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)  
    print("hsv",hsv.shape)
    channels.append(hsv[:,:,0])
    channels.append(hsv[:,:,1])
    print(len(channels))
    
    cmyk = pil_bgr_to_cmyk(bgr)
    print("cmyk",cmyk.shape)
    channels.append(cmyk[:,:,0])
    channels.append(cmyk[:,:,1])
    channels.append(cmyk[:,:,2])
    print(len(channels))
    
    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
    print("lab",lab.shape)
    channels.append(lab[:,:,1])
    channels.append(lab[:,:,2])
    print(len(channels))
    channels=[cv2.equalizeHist(ch) for ch in channels]
    
    return channels

In [6]:
img = imread_resized("wooden_folia_6", "0001.jpg", None)
cv2.imwrite("visualized/img.jpg", img)

True

In [7]:
channels=convert_to_gradient_channels(img)

for i in range(len(channels)):
    cv2.imwrite("visualized/channel_{}.jpg".format(i), channels[i])

(10, 240, 426)


In [8]:
channels=to_color_channels(img)

for i in range(len(channels)):
    cv2.imwrite("visualized/color_channel_{}.jpg".format(i), channels[i])

(240, 426, 3)
hsv (240, 426, 3)
2
cmyk (240, 426, 4)
5
lab (240, 426, 3)
7
